In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import re

import random
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Read in data set; change separators to '|' instead of ',' so that node lists don't get cut off

In [3]:
for_pd = StringIO()
with open('../data/accre-jobs-2020.csv') as accre:
    for line in accre:
        new_line = re.sub(r',', '|', line.rstrip(), count=12)
        print (new_line, file=for_pd)

for_pd.seek(0)

accre = pd.read_csv(for_pd, sep='|')#[1000000:1005000] getting the full data set
print (accre)

               JOBID        ACCOUNT       USER    REQMEM    USEDMEM      REQTIME     USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST
0           15925210        treviso   arabella  122880Mn  65973.49M  13-18:00:00  13-18:00:28      1    24  production      0:0  COMPLETED   cn1531
1           15861126        treviso   arabella  122880Mn  67181.12M  13-18:00:00  12-14:50:56      1    24  production      0:0  COMPLETED   cn1441
2           15861125        treviso   arabella  122880Mn  69111.86M  13-18:00:00  13-18:00:20      1    24  production      0:0  COMPLETED   cn1464
3           16251645        treviso   arabella  122880Mn  65317.33M  13-18:00:00  12-03:50:32      1    24  production      0:0  COMPLETED   cn1473
4           16251646        treviso   arabella  122880Mn  65876.11M  13-18:00:00  13-18:00:03      1    24  production      0:0  COMPLETED   cn1440
...              ...            ...        ...       ...        ...          ...          ...    ...   ...      

In [4]:
accre.tail()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312
3816287,24173815_10,portabella,vennie,32768Mn,21207.96M,1-00:00:00,00:01:40,1,2,production,0:0,COMPLETED,cn368
3816288,24173817,boysenberries,hortensia,100000Mn,0,1-00:00:00,00:03:05,1,8,production,0:0,RUNNING,cn1509
3816289,24173866,horseradish,ariane,20480Mn,0,05:00:00,00:02:05,1,1,production,0:0,RUNNING,cn1131


In [5]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   ACCOUNT    object
 2   USER       object
 3   REQMEM     object
 4   USEDMEM    object
 5   REQTIME    object
 6   USEDTIME   object
 7   NODES      int64 
 8   CPUS       int64 
 9   PARTITION  object
 10  EXITCODE   object
 11  STATE      object
 12  NODELIST   object
dtypes: int64(2), object(11)
memory usage: 378.5+ MB


In [6]:
accre.describe

<bound method NDFrame.describe of                JOBID        ACCOUNT       USER    REQMEM    USEDMEM      REQTIME     USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST
0           15925210        treviso   arabella  122880Mn  65973.49M  13-18:00:00  13-18:00:28      1    24  production      0:0  COMPLETED   cn1531
1           15861126        treviso   arabella  122880Mn  67181.12M  13-18:00:00  12-14:50:56      1    24  production      0:0  COMPLETED   cn1441
2           15861125        treviso   arabella  122880Mn  69111.86M  13-18:00:00  13-18:00:20      1    24  production      0:0  COMPLETED   cn1464
3           16251645        treviso   arabella  122880Mn  65317.33M  13-18:00:00  12-03:50:32      1    24  production      0:0  COMPLETED   cn1473
4           16251646        treviso   arabella  122880Mn  65876.11M  13-18:00:00  13-18:00:03      1    24  production      0:0  COMPLETED   cn1440
...              ...            ...        ...       ...        ...          .

In [7]:
accre.PARTITION.value_counts()

production              3311788
nogpfs                   327652
sam                       79151
pascal                    48004
turing                    39406
debug                      6738
maxwell                    3348
cgw-capra1                   83
cgw-dougherty1               72
cgw-cqs1                     34
cgw-vm-qa-flatearth1          4
cgw-cqs3                      4
devel                         2
cgw-rocksteady                2
cgw-tbi01                     2
Name: PARTITION, dtype: int64

In [8]:
accre.STATE.value_counts()

COMPLETED              3804644
CANCELLED                 6107
RUNNING                   3108
FAILED                    1406
CANCELLED by 686562        681
OUT_OF_MEMORY               94
TIMEOUT                     80
CANCELLED by 505355         63
CANCELLED by 855431         17
CANCELLED by 782611         13
CANCELLED by 9206            9
CANCELLED by 397600          7
CANCELLED by 200557          6
CANCELLED by 0               5
CANCELLED by 454080          4
CANCELLED by 199766          4
CANCELLED by 486541          4
CANCELLED by 9201            3
CANCELLED by 9202            3
CANCELLED by 649319          3
CANCELLED by 483348          3
CANCELLED by 124006          2
CANCELLED by 895426          2
CANCELLED by 693461          2
CANCELLED by 199066          2
CANCELLED by 90423           2
CANCELLED by 666860          2
CANCELLED by 546080          2
CANCELLED by 791651          2
CANCELLED by 515423          2
CANCELLED by 785271          1
CANCELLED by 651701          1
CANCELLE

Separate memory amount from memory units

In [9]:
test_split = accre.loc[1000000,'REQMEM']
test_split

'5000Mn'

In [10]:
re.findall(r'..$', test_split)
# also re.findall(r'\D+', test_split)

['Mn']

In [11]:
re.findall(r'\d+', test_split)

['5000']

Extract req memory quantity only and put it in a new column

In [12]:
accre['REQMEM_NO'] = accre['REQMEM'].str.extract('(\d+)')

In [13]:
accre.head(100)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,122880
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,122880
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,122880
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,122880
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,122880


Extract req memory units (type) only and put it in a new column

In [14]:
accre['REQMEM_TYPE'] = accre['REQMEM'].str.extract('(\D+)')

In [15]:
# convert column values to numeric values
accre['REQMEM_NO'] = pd.to_numeric(accre['REQMEM_NO'])

In [16]:
# isolate Mc req memory units only
accre[accre['REQMEM_TYPE'] == 'Mc']

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE
705,17013893_10,cep,codie,4096Mc,1725.41M,7-00:00:00,2-15:47:46,1,1,production,0:0,COMPLETED,cn421,4096,Mc
706,17013952_31,cep,codie,4096Mc,1733.61M,7-00:00:00,2-15:44:35,1,1,production,0:0,COMPLETED,cn1394,4096,Mc
2423,17043861_149,cep,sasha,1024Mc,375.44M,1-04:00:00,1-02:35:35,1,1,production,0:0,COMPLETED,cn1090,1024,Mc
2424,17043861_159,cep,sasha,1024Mc,375.38M,1-04:00:00,1-04:00:08,1,1,production,0:0,COMPLETED,cn1093,1024,Mc
2425,17043861_160,cep,sasha,1024Mc,375.32M,1-04:00:00,1-04:00:22,1,1,production,0:0,COMPLETED,cn1093,1024,Mc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816132,24173475,cms,lailah,1024Mc,62.46M,1-20:00:00,00:00:47,1,1,production,0:0,COMPLETED,cn394,1024,Mc
3816144,24173491,cmsadmin,autocms,4096Mc,869.54M,12:00:00,00:04:19,1,1,production,0:0,COMPLETED,cn421,4096,Mc
3816175,24173567,cms,lailah,1024Mc,1014.77M,1-20:00:00,00:05:20,1,1,production,0:0,COMPLETED,cn394,1024,Mc
3816215,24173661,cmsadmin,autocms,4096Mc,860.03M,12:00:00,00:03:50,1,1,production,0:0,COMPLETED,cn394,4096,Mc


In [17]:
accre.head(10)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880,Mn
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880,Mn
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880,Mn
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880,Mn
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880,Mn
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,122880,Mn
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,122880,Mn
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,122880,Mn
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,122880,Mn
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,122880,Mn


In [18]:
# testing out math for memory conversions
test_mn = accre.loc[1000000, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mn['CPUS'] = pd.to_numeric(1)
test_mn

REQMEM_NO        5000
REQMEM         5000Mn
NODES               1
CPUS                1
REQMEM_TYPE        Mn
Name: 1000000, dtype: object

In [19]:
test_mc = accre.loc[1000006, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mc['CPUS'] = pd.to_numeric(1)
test_mc

REQMEM_NO        2048
REQMEM         2048Mc
NODES               1
CPUS                1
REQMEM_TYPE        Mc
Name: 1000006, dtype: object

In [20]:
if test_mc['REQMEM_TYPE'] == 'Mc':
    print(test_mc['REQMEM_NO'])
elif test_mc['REQMEM_TYPE'] == 'Mn':
    print(test_mc['REQMEM_NO'] * (test_mc['NODES']/test_mc['CPUS']))

2048


In [21]:
if test_mn['REQMEM_TYPE'] == 'Mc':
    print(test_mn['REQMEM_NO'])
elif test_mn['REQMEM_TYPE'] == 'Mn':
    print(test_mn['REQMEM_NO'] * (test_mn['NODES']/test_mn['CPUS']))

5000.0


In [22]:
accre['REQMEM_PER_CORE'] = np.where((accre['REQMEM_TYPE'] == 'Mn'), \
                                      (accre['REQMEM_NO'] * (accre['NODES']/accre['CPUS'])), \
                                       accre['REQMEM_NO'])

accre.head(50)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.000000
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880,Mn,5120.000000
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.000000
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.000000
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880,Mn,5120.000000
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,122880,Mn,5120.000000
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.000000
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,122880,Mn,5120.000000
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.000000
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.000000


In [23]:
accre['USEDMEM_TYPE'] = accre['USEDMEM'].str.extract('(\D$)')

In [24]:
accre['USEDMEM_NO'] = accre['USEDMEM'].str.extract('(\d+)')

In [25]:
accre.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.0,M,65973
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880,Mn,5120.0,M,67181
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.0,M,69111
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.0,M,65317
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880,Mn,5120.0,M,65876


In [26]:
accre['USEDMEM_NO'] = pd.to_numeric(accre['USEDMEM_NO'])

In [27]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   JOBID            object 
 1   ACCOUNT          object 
 2   USER             object 
 3   REQMEM           object 
 4   USEDMEM          object 
 5   REQTIME          object 
 6   USEDTIME         object 
 7   NODES            int64  
 8   CPUS             int64  
 9   PARTITION        object 
 10  EXITCODE         object 
 11  STATE            object 
 12  NODELIST         object 
 13  REQMEM_NO        int64  
 14  REQMEM_TYPE      object 
 15  REQMEM_PER_CORE  float64
 16  USEDMEM_TYPE     object 
 17  USEDMEM_NO       int64  
dtypes: float64(1), int64(4), object(13)
memory usage: 524.1+ MB


In [28]:
accre['USEDMEM_PER_CORE'] = accre['USEDMEM_NO'] / accre['CPUS']

In [29]:
accre.head(10)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO,USEDMEM_PER_CORE
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.0,M,65973,2748.875000
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880,Mn,5120.0,M,67181,2799.208333
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.0,M,69111,2879.625000
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.0,M,65317,2721.541667
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880,Mn,5120.0,M,65876,2744.833333
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,122880,Mn,5120.0,M,65183,2715.958333
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.0,M,64941,2705.875000
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,122880,Mn,5120.0,M,68499,2854.125000
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.0,M,67761,2823.375000
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.0,M,68087,2836.958333


In [30]:
# time required and used; conversion to seconds
# Create a function to split the hh:mm:ss string and calculate seconds from it
def to_sec(x):
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

In [31]:
# Separate out the days from REQTIME, put them in another column, convert to a number, fill NaN with 0, convert to seconds
accre['REQTIME_DAY_SEC'] = accre['REQTIME'].str.extract('(\d+)-')
accre['REQTIME_DAY_SEC'] = pd.to_numeric(accre['REQTIME_DAY_SEC'])
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC'].fillna(0)
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC']*24*60*60
# Extract the hh:mm:ss from REQTIME, put in a new column, and then apply the to_sec function 
accre['REQTIME_T'] = accre['REQTIME'].str.extract('(..:..:..)$')
# REQTIME_SEC includes total seconds from REQTIME
accre['REQTIME_SEC'] = accre['REQTIME_T'].apply(to_sec) + accre['REQTIME_DAY_SEC']
# Do the same for USEDTIME
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME'].str.extract('(\d+)-')
accre['USEDTIME_DAY_SEC'] = pd.to_numeric(accre['USEDTIME_DAY_SEC'])
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC'].fillna(0)
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC']*24*60*60
# Do the same for USEDTIME
accre['USEDTIME_T'] = accre['USEDTIME'].str.extract('(..:..:..)$')
# USEDTIME_SEC includes total second from USEDTIME
accre['USEDTIME_SEC'] = accre['USEDTIME_T'].apply(to_sec) + accre['USEDTIME_DAY_SEC']
# Check to make sure the data types look okay
accre.head(20)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO,USEDMEM_PER_CORE,REQTIME_DAY_SEC,REQTIME_T,REQTIME_SEC,USEDTIME_DAY_SEC,USEDTIME_T,USEDTIME_SEC
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.0,M,65973,2748.875000,1123200.0,18:00:00,1188000.0,1123200.0,18:00:28,1188028.0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,122880,Mn,5120.0,M,67181,2799.208333,1123200.0,18:00:00,1188000.0,1036800.0,14:50:56,1090256.0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.0,M,69111,2879.625000,1123200.0,18:00:00,1188000.0,1123200.0,18:00:20,1188020.0
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.0,M,65317,2721.541667,1123200.0,18:00:00,1188000.0,1036800.0,03:50:32,1050632.0
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,122880,Mn,5120.0,M,65876,2744.833333,1123200.0,18:00:00,1188000.0,1123200.0,18:00:03,1188003.0
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,122880,Mn,5120.0,M,65183,2715.958333,1123200.0,18:00:00,1188000.0,1036800.0,02:31:43,1045903.0
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,122880,Mn,5120.0,M,64941,2705.875000,1123200.0,18:00:00,1188000.0,1123200.0,06:54:59,1148099.0
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,122880,Mn,5120.0,M,68499,2854.125000,1123200.0,18:00:00,1188000.0,1123200.0,18:00:05,1188005.0
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,122880,Mn,5120.0,M,67761,2823.375000,1123200.0,18:00:00,1188000.0,1123200.0,18:00:15,1188015.0
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,122880,Mn,5120.0,M,68087,2836.958333,1123200.0,18:00:00,1188000.0,1123200.0,18:00:06,1188006.0


In [32]:
accre.to_csv(r'../data/accre-jobs-2020-clean.csv', index = False)

In [33]:
# breaking up node lists
nodelist = failed['NODELIST'].unique().tolist()
nodelist_nums = [i.strip('cn').strip('[').strip(']') for i in nodelist]
for n in nodelist_nums:
    if ',' in n: 
        nodes = n.split(',')
        if '-' in n:
            nodes = n.split('-')
            low, high = int(nodes[0]), int(nodes[1])       
        else:
            nodes = int(n)


NameError: name 'failed' is not defined